*Project 1 - Machine Learning for NLP*

*Michele Pulvirenti, Marco Riva*

In [56]:
!pip install fast-langdetect

# Project 1 - TripAdvisor reccomandation system

## 1. Data preparation

### Download dataset

In [57]:
import kagglehub
import zipfile
import os

dataset_filename = 'reviews.csv'
cache_dataset_zipped = kagglehub.dataset_download("joebeachcapital/hotel-reviews", path=dataset_filename)
# the file is actually downloaded as zipped, even if there is not '.zip' extensions
# so we need to extract it
extract_path = './'
# Check if the file already exists
if not os.path.exists(extract_path + dataset_filename):
    with zipfile.ZipFile(cache_dataset_zipped, 'r') as zip_ref:
        # Iterate through each file in the zip file
        for file in zip_ref.namelist():
            # Check if the file already exists
            if not os.path.exists(os.path.join(extract_path, file)):
                # Extract the file
                zip_ref.extract(file, extract_path)
else:
    print(f"File '{dataset_filename}' already exists")

dataset_path = extract_path + dataset_filename

File 'reviews.csv' already exists


### Import dataset

In [58]:
import pandas as pd

df = pd.read_csv(dataset_path)
df.head()

,ratings,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile
0,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...","“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,"{'username': 'Papa_Panda', 'num_cities': 22, '...",December 2012,93338,0,2012-12-17,147643103,False
1,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...","{'username': 'Maureen V', 'num_reviews': 2, 'n...",December 2012,93338,0,2012-12-17,147639004,False
2,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Great Stay”,This is a great property in Midtown. We two di...,"{'username': 'vuguru', 'num_cities': 12, 'num_...",December 2012,1762573,0,2012-12-18,147697954,False
3,"{'service': 5.0, 'cleanliness': 5.0, 'overall'...",“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,"{'username': 'Hotel-Designer', 'num_cities': 5...",August 2012,1762573,0,2012-12-17,147625723,False
4,"{'service': 4.0, 'cleanliness': 5.0, 'overall'...",“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,"{'username': 'JamesE339', 'num_cities': 34, 'n...",December 2012,1762573,0,2012-12-17,147612823,False


## 2. Data formatting

### Flatten the json 'ratings' column

The values of **ratings** are json object as string.

In [59]:
from pandas import json_normalize
# convert JSON column to DataFrame
json_df = json_normalize(df.ratings.apply(eval))

# merge the two dataframes
df = df.drop(columns=['ratings']).join(json_df)
df

,title,text,author,date_stayed,offering_id,num_helpful_votes,date,id,via_mobile,service,cleanliness,overall,value,location,sleep_quality,rooms,check_in_front_desk,business_service_(e_g_internet_access)
0,"“Truly is ""Jewel of the Upper Wets Side""”",Stayed in a king suite for 11 nights and yes i...,"{'username': 'Papa_Panda', 'num_cities': 22, '...",December 2012,93338,0,2012-12-17,147643103,False,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
1,“My home away from home!”,"On every visit to NYC, the Hotel Beacon is the...","{'username': 'Maureen V', 'num_reviews': 2, 'n...",December 2012,93338,0,2012-12-17,147639004,False,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
2,“Great Stay”,This is a great property in Midtown. We two di...,"{'username': 'vuguru', 'num_cities': 12, 'num_...",December 2012,1762573,0,2012-12-18,147697954,False,4.0,5.0,4.0,4.0,5.0,4.0,4.0,NaN,NaN
3,“Modern Convenience”,The Andaz is a nice hotel in a central locatio...,"{'username': 'Hotel-Designer', 'num_cities': 5...",August 2012,1762573,0,2012-12-17,147625723,False,5.0,5.0,4.0,5.0,5.0,5.0,5.0,NaN,NaN
4,“Its the best of the Andaz Brand in the US....”,I have stayed at each of the US Andaz properti...,"{'username': 'JamesE339', 'num_cities': 34, 'n...",December 2012,1762573,0,2012-12-17,147612823,False,4.0,5.0,4.0,3.0,5.0,5.0,5.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878556,“vil komme igen”,"Pænt hotel i et livligt miljø. Store værelser,...","{'username': '', 'id': '', 'location': ''}",NaN,84093,0,2008-08-31,51441576,False,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
878557,“excellent”,"un excellent hotel 4 *, pour un prix raisonnab...","{'username': 'dan016', 'num_reviews': 2, 'num_...",July 2008,84093,0,2008-07-18,18003332,False,4.0,5.0,4.0,4.0,4.0,NaN,5.0,5.0,NaN
878558,“Un hotel eccezionale”,"L'hotel, situato in una zona tranquilla e a du...","{'username': '', 'id': '', 'location': ''}",July 2008,84093,0,2008-04-18,15564515,False,5.0,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN
878559,“Gerne wieder”,Sehr schön ausgestattetes Hotel in bester Lage...,"{'username': '', 'id': '', 'location': ''}",July 2008,84093,0,2008-04-01,15564508,False,5.0,5.0,5.0,NaN,NaN,NaN,5.0,NaN,NaN


### Remove unused columns and rows

In order to compare places accurately, we keep only the reviews with ratings only on relevant criteria, not more and not less.

In [60]:
# unused columns
df.drop(columns=['title', 'author', 'date_stayed', 'num_helpful_votes', 'date', 'id', 'via_mobile'], inplace=True, errors='ignore')
# reviews that have null values on relevant rating criteria
relevant_rating_columns = ['service', 'cleanliness', 'overall', 'value', 'location', 'sleep_quality', 'rooms']
df.dropna(subset=relevant_rating_columns, inplace=True)
# reviews that have more rating criteria than the relevant ones
df = df[~df['check_in_front_desk'].notna()]
df = df[~df['business_service_(e_g_internet_access)'].notna()]
df.drop(columns=['check_in_front_desk', 'business_service_(e_g_internet_access)'], inplace=True, errors='ignore')
df

,text,offering_id,service,cleanliness,overall,value,location,sleep_quality,rooms
0,Stayed in a king suite for 11 nights and yes i...,93338,5.0,5.0,5.0,5.0,5.0,5.0,5.0
1,"On every visit to NYC, the Hotel Beacon is the...",93338,5.0,5.0,5.0,5.0,5.0,5.0,5.0
2,This is a great property in Midtown. We two di...,1762573,4.0,5.0,4.0,4.0,5.0,4.0,4.0
3,The Andaz is a nice hotel in a central locatio...,1762573,5.0,5.0,4.0,5.0,5.0,5.0,5.0
4,I have stayed at each of the US Andaz properti...,1762573,4.0,5.0,4.0,3.0,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...
878548,"Das Hotel Palomar gehoert zur Kimpton Gruppe, ...",84093,4.0,5.0,4.0,3.0,4.0,4.0,4.0
878549,Anlässlich einer Geschäftsreise waren wir das ...,84093,5.0,5.0,5.0,5.0,5.0,5.0,5.0
878550,"Das Hotel liegt in Arlington, eine etwas gehob...",84093,5.0,5.0,5.0,4.0,5.0,5.0,5.0
878551,Mon mari et moi sommes restés à l'hôtel du jeu...,84093,5.0,5.0,5.0,5.0,5.0,5.0,5.0


## 3. Data processing

To obtain better results, some preprocessing techniques are applied such as: language detection, tokenization, lowercasing, removing puntualization, speling correction, stopwords...

### Language detection

We have to check the language of all reviews.

#### Inspect

In [61]:
# ensures no empty strings
print('empty reviews count:', df.text.apply(lambda x: x.strip() == '').sum())
df.isnull().sum()

empty reviews count: 0


text             0
offering_id      0
service          0
cleanliness      0
overall          0
value            0
location         0
sleep_quality    0
rooms            0
dtype: int64

In [62]:
# assess the reviews are mostly long, therefore helping in language detection
short_len_def = 70
short_reviews_percentage = round((len(df[df['text'].str.len() < short_len_def]) / len(df)) * 100, 2)
print(f'short (less than {short_len_def} character long) reviews: {short_reviews_percentage}%')

short (less than 70 character long) reviews: 0.31%


#### Detect

For detecting reviews in other languages than English we tried two libraries:
- `langdetect`
- `fast-langdetect`

We chose to use the second one because it gave the results 8x faster than the first.

As is written in the docs:

*" fast-langdetect provides ultra-fast and highly accurate language detection based on FastText, a library developed by Facebook. This package is 80x faster than traditional methods and offers 95% accuracy "*

Here is the version using `langdetect`

```python
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

# ensure consistent results
DetectorFactory.seed = 0

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return 'unknown'
```

In [63]:
from fast_langdetect import detect

def detect_language(text):
    try:
        return detect(text.replace("\n", ""), low_memory=False)["lang"]
    except Exception:
        print(Exception.message)
        return 'unknown'

if 'language' not in df.columns:
    df['language'] = df['text'].apply(detect_language)

df

,text,offering_id,service,cleanliness,overall,value,location,sleep_quality,rooms,language
0,Stayed in a king suite for 11 nights and yes i...,93338,5.0,5.0,5.0,5.0,5.0,5.0,5.0,en
1,"On every visit to NYC, the Hotel Beacon is the...",93338,5.0,5.0,5.0,5.0,5.0,5.0,5.0,en
2,This is a great property in Midtown. We two di...,1762573,4.0,5.0,4.0,4.0,5.0,4.0,4.0,en
3,The Andaz is a nice hotel in a central locatio...,1762573,5.0,5.0,4.0,5.0,5.0,5.0,5.0,en
4,I have stayed at each of the US Andaz properti...,1762573,4.0,5.0,4.0,3.0,5.0,5.0,5.0,en
...,...,...,...,...,...,...,...,...,...,...
878548,"Das Hotel Palomar gehoert zur Kimpton Gruppe, ...",84093,4.0,5.0,4.0,3.0,4.0,4.0,4.0,de
878549,Anlässlich einer Geschäftsreise waren wir das ...,84093,5.0,5.0,5.0,5.0,5.0,5.0,5.0,de
878550,"Das Hotel liegt in Arlington, eine etwas gehob...",84093,5.0,5.0,5.0,4.0,5.0,5.0,5.0,de
878551,Mon mari et moi sommes restés à l'hôtel du jeu...,84093,5.0,5.0,5.0,5.0,5.0,5.0,5.0,fr


In [64]:
# Calculate the percentage of each language
language_counts = df['language'].value_counts(normalize=True) * 100
# language_counts.rename('percentage', inplace=True).to_dict()
language_counts.to_dict()

{'en': 90.37849829038674,
 'de': 2.4154589371980677,
 'it': 2.0749113109479413,
 'fr': 1.8663659947382412,
 'es': 1.6433829258678694,
 'ja': 0.6909495916178533,
 'pt': 0.5225091439100185,
 'nl': 0.11298114383668381,
 'zh': 0.11091860774230217,
 'sv': 0.08043890768088442,
 'da': 0.03391726021872049,
 'ru': 0.026812969226961473,
 'no': 0.02497960380973334,
 'tr': 0.007104290991759022,
 'pl': 0.004125072188763303,
 'el': 0.0022917067715351683,
 'ko': 0.0013750240629211011,
 'th': 0.0009166827086140674,
 'id': 0.0006875120314605506,
 'hr': 0.00022917067715351686,
 'ar': 0.00022917067715351686,
 'ca': 0.00022917067715351686,
 'hu': 0.00022917067715351686,
 'fa': 0.00022917067715351686,
 'nn': 0.00022917067715351686}

In [65]:
# visual quality check (second most common language)
second_common = list(language_counts.keys())[1]
df[df['language'] == second_common].head()

,text,offering_id,service,cleanliness,overall,value,location,sleep_quality,rooms,language
1066,Optimaler Aufenthalt für 3 Tage! Traumhaftes B...,239853,5.0,5.0,5.0,4.0,5.0,5.0,5.0,de
1107,Das Hotel liegt optimal nach Downtown (incl. Z...,239853,1.0,4.0,4.0,3.0,4.0,4.0,4.0,de
1110,Wir hatten nur eine Übernachtung in Houston bi...,239853,4.0,5.0,4.0,4.0,5.0,4.0,4.0,de
1151,Wir waren im März 2010 für ein paar Tage in Ho...,239853,4.0,5.0,4.0,4.0,5.0,5.0,5.0,de
1494,Es handelt sich um ein neues tolles Haus mit K...,1966350,4.0,4.0,4.0,4.0,5.0,5.0,5.0,de


#### Filter
Since the dataset is mostly in English, we can filter out the non-English reviews.

In [66]:
# keep only english reviews
df = df[df['language'] == 'en']
df = df.drop(columns=['language'])

#### Different approach

Here is the version using lists of words and characters to filter out non-English reviews.
DESCRIZIONE

```python
# Approccio diverso per rilevare la lingua, ma non è molto efficiente
# per parole
to_filter = ['bonne','de','le','les','une','des','du','ce','cet','cette','ces','mon','ma','mes','ton','ta','tes','sa','ses','notre','nos','votre','vos','leur','leurs','quel','quelle','quels','quelles','ceci','cela','celui','celle','ceux','celles']
pattern = r'\b(?:' + '|'.join(to_filter) + r')\b'
# per caratteri
to_filter = ['â','ä','ç','é','è','ê','ë','î','ï','ô','ö','ù','û','ü','ÿ']
pattern = '|'.join(to_filter)
# Filter the DataFrame
df_filtered = df[df['text'].str.contains(pattern, case=False, na=False, regex=True)]

# Display the resulting DataFrame
df_filtered
```

### Group by offering_id

In [67]:
df = df.groupby('offering_id').agg({
    'text': ' '.join,
    'service': 'mean',
    'cleanliness': 'mean',
    'overall': 'mean',
    'value': 'mean',
    'location': 'mean',
    'sleep_quality': 'mean',
    'rooms': 'mean'
}).reset_index()

# Display the resulting DataFrame
df

,offering_id,text,service,cleanliness,overall,value,location,sleep_quality,rooms
0,72572,I had to make fast visit to seattle and I foun...,4.604061,4.634518,4.390863,4.324873,4.568528,4.329949,4.284264
1,72579,"Great service, rooms were clean, could use som...",4.247934,4.264463,3.892562,4.148760,4.206612,3.776860,3.876033
2,72586,Beautiful views of the space needle - especial...,4.261905,4.301587,4.071429,4.087302,4.587302,4.119048,4.015873
3,72598,This hotel is in need of some serious updates....,3.243243,3.243243,2.918919,3.054054,3.027027,3.270270,3.189189
4,73236,My experience at this days inn was perfect. th...,4.277778,3.111111,3.388889,3.777778,4.111111,3.722222,3.222222
...,...,...,...,...,...,...,...,...,...
3722,3513445,"New rooms, clean beds, bathroom. You have wifi...",3.666667,4.000000,3.666667,4.000000,3.000000,4.000000,3.666667
3723,3523356,I've stayed at plenty of Hampton Inns during m...,4.928571,4.928571,4.571429,4.214286,4.500000,4.571429,4.500000
3724,3541823,"Inn staff absolutely wonderful, helpful, knowl...",4.750000,4.500000,4.000000,4.500000,5.000000,3.750000,3.250000
3725,3572384,"Crowded, noisy, dirty. Service is poor, food i...",3.000000,2.000000,2.000000,2.000000,4.000000,3.000000,2.000000


## 4. Tokenization, lowercasing, stopwords and puntualization removal, spellchecking??, lemmatization

In [68]:
import nltk
from nltk.corpus import stopwords
# from spellchecker import SpellChecker
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
lemmatizer = WordNetLemmatizer()

stop_words = set(stopwords.words('english'))

def preprocess(text):
    words = word_tokenize(text.lower())
    # isalnum allows us to remove punctuation
    filtered_words = [word for word in words if word.isalnum() and word not in stop_words]
    # spellechecking ??
    # spell = SpellChecker()
    # word = [spell.correction(word) if word not in spell else word for word in word_list]
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    return lemmatized_words

[nltk_data] Downloading package punkt_tab to /Users/Marco/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Marco/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/Marco/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
df['text'] = df['text'].apply(preprocess)
df

,offering_id,text,service,cleanliness,overall,value,location,sleep_quality,rooms
0,72572,"[make, fast, visit, seattle, found, pioneer, r...",4.604061,4.634518,4.390863,4.324873,4.568528,4.329949,4.284264
1,72579,"[great, service, room, clean, could, use, quee...",4.247934,4.264463,3.892562,4.148760,4.206612,3.776860,3.876033
2,72586,"[beautiful, view, space, needle, especially, n...",4.261905,4.301587,4.071429,4.087302,4.587302,4.119048,4.015873
3,72598,"[hotel, need, serious, update, room, big, bed,...",3.243243,3.243243,2.918919,3.054054,3.027027,3.270270,3.189189
4,73236,"[experience, day, inn, perfect, staff, great, ...",4.277778,3.111111,3.388889,3.777778,4.111111,3.722222,3.222222


## 11. TRAINING

### First attempt at implementing baseline model

In [192]:
def random_offering(df):
    return df.sample()['offering_id'].values[0]

def offering_index(df, offering_id):
    if offering_id not in df['offering_id'].values:
        RuntimeError(f"id {offering_id} not found. Is this the correct dataset?")
    return df[df['offering_id'] == offering_id].index[0]
    
def drop_tokenize_df(df, offering_id):
    df_edited = df.drop(index=offering_index(df, offering_id), inplace=False)
    return df_edited['text'].tolist(), df_edited

def offering_text(df, offering_id):
    return df['text'][offering_index(df, offering_id)]
def offering_ratings(df, offering_id):
    return df.iloc[offering_index(df, offering_id)][relevant_rating_columns]

def retrieve_best_offering(df, scores):
    if len(df) != len(scores):
        RuntimeError("Wrong dataset (len of df and scores differ)")
    return df.iloc[np.argmax(scores)]['offering_id']

query_offering_id = df['offering_id'][0]

In [193]:
from rank_bm25 import BM25Okapi
import numpy as np

# Prepare the BM25 model
tokenized_corpus, df_train = drop_tokenize_df(df, query_offering_id)
bm25 = BM25Okapi(tokenized_corpus)

# Compute BM25 scores
scores = bm25.get_scores(offering_text(df, query_offering_id))

# Get the index of the most relevant document
best_offering = retrieve_best_offering(df_train, scores)
print(f"Most similar place to {query_offering_id} is {best_offering}")

Most similar place to 72572 is 677260


### BM25 documentation example

from rank_bm25 import BM25Okapi

corpus = [
    "Hello there good man!", # documento 1
    "It is quite windy in London", # documento 2
    "How is the weather today?" # documento 3
]

# in input ci deve essere una lista di liste di stringhe
model_input = [doc.split(" ") for doc in corpus]
print(model_input)
tokenized_corpus = model_input

bm25 = BM25Okapi(tokenized_corpus)
# per recuperare i best n documenti
bm25.get_top_n(tokenized_query, corpus, n=1)

## 12. VALIDATION

In [269]:
from sklearn.metrics import mean_squared_error

def offering_mse(df_q, query_offering, result_offering, df_r=None):
    if df_r is None:
        df_r = df_q
    return mean_squared_error(offering_ratings(df_q, query_offering), offering_ratings(df_r, result_offering))

def mse_to_stars(mse):
    return (4 * mse) / 16

def mse_to_error_percentage(mse):
    return (mse / 4) * 100

def normalized_offering_mse(df_q, query_offering, result_offering, df_r=None):
    result_mse = offering_mse(df_q, query_offering, result_offering, df_r)
    
    # Compute MSEs for all other places
    if df_r is None:
        df_r = df_q.drop(index=offering_index(df_q, query_offering), inplace=False).reset_index(drop=True)
    
    mses = [offering_mse(df_q, query_offering, df_r['offering_id'][i], df_r) for i in range(len(df_r))]

    # Normalize the MSE for the predicted place
    normalization_factor = np.mean(mses)
    normalized_mse = result_mse / normalization_factor
    return normalized_mse, normalization_factor

def pretty_evaluate(df_q, query_offering, result_offering, df_r=None):
    mse = offering_mse(df, query_offering_id, best_offering)
    mse_normalized, normalization_factor = normalized_offering_mse(df, query_offering_id, best_offering)
    print(f"""
MSE:            {round(mse,5):.5f}   (error:    {round(mse_to_error_percentage(mse),2):.2f} %)
MSE normalized: {round(mse_normalized,5):.5f}   (avg MSE: {round(normalization_factor,5):.5f})
""")

In [270]:
pretty_evaluate(df, query_offering_id, best_offering)


MSE:            0.02739   (error:    0.68 %)
MSE normalized: 0.02380   (avg MSE: 1.15050)

